In [7]:
#Program scrapping FAKE and REAL news data from POLITIFACT.com and use as Dataset. 

In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [9]:

authors = []
dates = []
statements = []
sources = []
targets = []

In [10]:
def scrape_website(page_number):
    page_num = str(page_number)
    URL = 'https://www.politifact.com/factchecks/list/?page='+page_num
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.text, 'html.parser')

    statement_footer = soup.find_all('footer', attrs={'class':'m-statement__footer'})
    statement_quote = soup.find_all('div', attrs= {'class': 'm-statement__quote'})
    statement_meta = soup.find_all('div', attrs={'class': 'm-statement__meta'}) 
    target = soup.find_all('div', attrs={'class':'m-statement__meter'})

    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name + ' '+ last_name
        month = name_and_date[4]
        day = name_and_date[5]
        year = name_and_date[6]
        date = month+' '+day+' '+year
        dates.append(date)
        authors.append(full_name)

    for i in statement_quote:
        link2 = i.find_all('a')
        statement_text = link2[0].text.strip()
        statements.append(statement_text)

    for i in statement_meta:
        link3 = i.find_all('a')
        source_text = link3[0].text.strip()
        sources.append(source_text)

    for i in target: 
        link4 = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(link4)

In [11]:
n = 2
for i in range(1,n):
    scrape_website(i)

In [12]:
data = pd.DataFrame(columns=['author', 'statement', 'source', 'date', 'target'])
data['author'] = authors
data['statement'] = statements
data['source'] = sources
data['date'] = dates
data['target'] = targets

In [13]:
data

,author,statement,source,date,target
0,Sofia Ahmed,“The population in Gaza has increased by 2.02%...,Instagram posts,"December 6, 2024",false
1,Madison Czopek,"Elon Musk wrote on X that “without CEOs, the w...",Threads posts,"December 6, 2024",false
2,Jeff Cercone,"""Vaccine researcher Peter Hotez says multiple ...",Instagram posts,"December 6, 2024",false
3,Sara Swann,Fox News commentator Dr. Kelly Powers died fro...,Facebook posts,"December 6, 2024",false
4,Loreben Tuquero,“Trump was impeached for trying to investigate...,Instagram posts,"December 6, 2024",false
5,Kwasi Gyamfi,“Elon Musk is not a citizen of the United Stat...,Threads posts,"• December 5,",false
6,Marta Campabadal,"""Última hora: secuestraron al doctor Juan J. R...",Facebook posts,"• December 5,",pants-fire
7,Sara Swann,Photo shows President Joe Biden used his son H...,Facebook posts,"December 5, 2024",false
8,Loreben Tuquero,“Bill Gates has been indicted in the Netherlan...,Instagram posts,"December 5, 2024",false
9,Ciara O'Rourke,“Elon Musk decides to withdraw support” for Wo...,Facebook posts,"December 5, 2024",false


In [14]:
def getBinaryNumTarget(text):
    if text == 'true':
        return '1'
    else:
        return '0'

In [17]:
def getBinaryTarget(text):
    if text == 'true':
        return 'REAL'
    else:
        return 'FAKE'

In [18]:
data['BinaryTarget']= data['target'].apply(getBinaryTarget)
data['BinaryNumTarget'] = data['target'].apply(getBinaryNumTarget)

In [19]:
data

,author,statement,source,date,target,BinaryTarget,BinaryNumTarget
0,Sofia Ahmed,“The population in Gaza has increased by 2.02%...,Instagram posts,"December 6, 2024",false,FAKE,0
1,Madison Czopek,"Elon Musk wrote on X that “without CEOs, the w...",Threads posts,"December 6, 2024",false,FAKE,0
2,Jeff Cercone,"""Vaccine researcher Peter Hotez says multiple ...",Instagram posts,"December 6, 2024",false,FAKE,0
3,Sara Swann,Fox News commentator Dr. Kelly Powers died fro...,Facebook posts,"December 6, 2024",false,FAKE,0
4,Loreben Tuquero,“Trump was impeached for trying to investigate...,Instagram posts,"December 6, 2024",false,FAKE,0
5,Kwasi Gyamfi,“Elon Musk is not a citizen of the United Stat...,Threads posts,"• December 5,",false,FAKE,0
6,Marta Campabadal,"""Última hora: secuestraron al doctor Juan J. R...",Facebook posts,"• December 5,",pants-fire,FAKE,0
7,Sara Swann,Photo shows President Joe Biden used his son H...,Facebook posts,"December 5, 2024",false,FAKE,0
8,Loreben Tuquero,“Bill Gates has been indicted in the Netherlan...,Instagram posts,"December 5, 2024",false,FAKE,0
9,Ciara O'Rourke,“Elon Musk decides to withdraw support” for Wo...,Facebook posts,"December 5, 2024",false,FAKE,0


In [20]:
data.to_csv('FactChecker_Dataset.csv')